In [1]:
#import image_analysis as ia
import NearestNeighbor as NN
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

# 1. Image Analysis
Image analysis involves 2 main steps: Segmentation and Object detection
## 1.1 Segmentation
First the images are segemented into individual cells and their boundaries are stored as ImageJ ROIs in zip files. These can be opened later in FIJI/ImageJ vor viewing and analysis.
Segmentation involves the following steps:
1. Thresholding: Images are thresholded by the multiotsu thresholding technique, which produces 2 numbers. The lower number is chosen as a threshold.
2. Cell separation: During thresholding cells may be too close together which would merge them together. To solve this problem Cell centers are identified and starting from the cell centers watershed segmentation is performed. This seperates the thrsholded area into individual labeled regions that represent the cells. Cell centers are identified via either of the following methods:
    - By nuclei: If nuclei were imaged by a DAPI stain, they are identified as cell centers.
    - By distance transform: If no nuclei were captured a distance transform is performed. Resulting peaks are taken as cell centers.
3. Cell identification and measurement: From each cell measurements are taken involving the area, mean and median intensities. Furthermore the boundaries are converted to ROIs and stored in a corresponding zip file.

In [2]:
data = "E:\Raphael\Data"
exID = 3
CD={0:"aSyn-GFP",1:"aSyn-AB",2:"LAMP1/LC3"}
#seg = ia.segmentation(data=data,exID=exID,p_max=500,t_mod=1,ch_dic=CD)
#seg.plot_segmentation(IDs=[7,32,77])
    

In [3]:
#seg.segment_all()

In [4]:
#detect=ia.detection(data=data,exID=exID,ch_dic=CD)
#detect.plot_detection(channel="aSyn-GFP")

In [5]:
#detect.detect_all()

In [6]:
nn = NN.NN(data=data,YChannel="aSyn-AB")

In [7]:
#dic = nn.calculate()

In [8]:
#nn.load()

In [9]:
#nn.save()
nn.load()

Opened from: E:\Raphael\Data\3. Plasmid moves cargo into Lysosomes\NN.p


In [10]:
df2 = nn.get_pval(t=0.5)
df2[df2["Group2"]=="LAMP1"].head()

E:\Raphael\LabBook\NearestNeighbor.py:180: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df2 = self.df.groupby(["Group2","Group1"])["N_D","N_X","N_Q","D","Q"].sum().reset_index()


,Group2,Group1,N_D,N_X,N_Q,D,Q,Ct,Co,e,crit_e,CtX,CoX,p,ci_lo,ci_hi
0,LAMP1,FY,404291,103183,12872680,108940,1660156,0.269459,0.128967,0.912753,0.918575,0.442796,0.317412,0.000000,0.905793,0.919690
1,LAMP1,TP,324743,90135,12867950,97535,1789754,0.300345,0.139086,0.977246,0.983539,0.442796,0.317412,0.000000,0.969732,0.984738
2,LAMP1,X,226034,201474,8769250,100087,2783467,0.442796,0.317412,0.535869,0.542823,0.442796,0.317412,0.499123,0.527562,0.544167


In [92]:
df = nn.get_pval_indiv(t=0.3)

In [93]:
df = df[df["Group2"]=="LAMP1"]
df.groupby(["Group1"])["e"].mean()

Group1
FY    1.044000
TP    0.291845
X     0.572727
Name: e, dtype: float64

In [94]:
group = df.groupby(["Group1"])["e"]
group.mean()+1.96*group.std()/np.sqrt(group.count())

Group1
FY    1.367440
TP    1.256305
X     0.797611
Name: e, dtype: float64

In [95]:
group.mean()-1.96*group.std()/np.sqrt(group.count())

Group1
FY    0.720560
TP   -0.672616
X     0.347844
Name: e, dtype: float64

In [96]:
df[df["p"]<0.05].groupby(["Group1"])["p"].count()/df.groupby(["Group1"])["p"].count()

Group1
FY    0.931034
TP    0.655172
X     0.777778
Name: p, dtype: float64

In [97]:
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
fvalue, pvalue = stats.f_oneway(df[df["Group1"]=="X"]['e'], df[df["Group1"]=="TP"]['e'], df[df["Group1"]=="FY"]['e'])
print(fvalue, pvalue)

1.3645575118378852 0.26193239501062154


In [98]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

m_comp = pairwise_tukeyhsd(endog=df['e'], groups=df['Group1'], alpha=0.05)
m_comp.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
FY,TP,-0.7522,0.2357,-1.8497,0.3454,False
FY,X,-0.4713,0.6426,-1.7253,0.7827,False
TP,X,0.2809,0.854,-0.9731,1.5349,False


In [56]:
bins = np.arange(0, np.max(dic[2]["Q"]),0.1)
Dhist = np.histogram(dic[2]["D"],bins=bins,density=True)[0]
Qhist = np.histogram(dic[2]["Q"],bins=bins,density=True)[0]
PQhist = Qhist*np.exp(-Dhist)
PQhist = PQhist/np.sum(PQhist)


NameError: name 'dic' is not defined

In [ ]:
import seaborn as sns

In [ ]:
sns.boxplot(data=df[df["Group2"]=="LC3"],x="Group1",y="N_X")

In [ ]:
sns.boxplot(data=df[df["Group2"]=="LAMP1"],x="Group1",y="intensity_mean_GFP")

In [ ]:
from math import log
def C(D,N):
    return D/N

def e(Ct,Co):
    #print(Ct)
    Ct = Ct.replace(to_replace = 0, value = 0.00000001)
    Co = Co.replace(to_replace = 0, value = 0.00000001)
    return np.log(Ct/(1-Ct))-np.log(Co/(1-Co))   

In [ ]:
df2 = df.groupby(["Group2","Group1"]).sum()
df2 = df2.reset_index()
df2.head()

In [ ]:
df2["Ct"] = C(df2["D"],df2["N_D"])
df2["Co"] = C(df2["Q"],df2["N_Q"])
df2["e"] = e(df2["Ct"],df2["Co"])
df2.head()

In [ ]:
from scipy.stats import binom

def crit_e(N,Ct,Co):
    CC = binom.ppf(0.95, N, Ct)
    print(C(CC,N))
    return e(C(CC,N),Co)

df2["crit_e"] = crit_e(df2["N_D"],df2["Ct"],df2["Co"])
df2.head()
    

In [ ]:
def p_val_step(e,N,CtX,CoX):
    E = np.exp(e+np.log(CoX/(1-CoX)))
    Ct = E/(1+E)
    p = 1-binom.cdf(Ct*N, N, CtX)
    return p

def confidence(N,Ct,Co):
    CI_Ct = binom.interval(0.95,N,Ct)
    CI = [e(ci/N,Co) for ci in CI_Ct]
    return CI[0],CI[1]

Control = "X"
df2["CtX"] = df2.apply(lambda row: pd.Series([df2["Ct"][df2["Group2"]==row["Group2"]][df2["Group1"]==Control].values[0]]), axis=1)
df2["CoX"] = df2.apply(lambda row: pd.Series([df2["Co"][df2["Group2"]==row["Group2"]][df2["Group1"]==Control].values[0]]), axis=1)
df2["p"] = p_val_step(df2["e"],df2["N_D"],df2["CtX"],df2["CoX"])
df2["ci_lo"],df2["ci_hi"] = confidence(df2["N_D"],df2["Ct"],df2["Co"])
df2.head()

In [ ]:
df["Ct"] = C(df["D"],df["N_D"])
df["Co"] = C(df["Q"],df["N_Q"])
df["e"] = e(df["Ct"],df["Co"])
df["p"] = p_val_step(df["e"],df["N_D"],df["Co"],df["Co"])
df["ci_lo"],df["ci_hi"] = confidence(df["N_D"],df["Ct"],df["Co"])
df.head()

In [ ]:
df[df["p"]<0.000001].groupby(["Group2","Group1"])["e"].count()/df.groupby(["Group2","Group1"])["e"].count()

In [ ]:
sns.swarmplot(data=df[df["Group2"]=="LC3"], x="Group1",y="e")

In [ ]:
df[df["e"]<10].groupby(["Group2","Group1"])["e"].count()/df.groupby(["Group2","Group1"])["e"].count()

In [ ]:
df[df["e"]>10].groupby(["Group2","Group1"])["e"].count()/df.groupby(["Group2","Group1"])["e"].count()

In [ ]:
df[df["e"]>10].groupby(["Group2","Group1"])["e"].median()

In [ ]:
from sklearn.mixture import GaussianMixture
GM = GaussianMixture(n_components=2).fit(df["e"].values.reshape(-1,1))
df["e_g"] = GM.predict(df["e"].values.reshape(-1,1))
df.groupby(["Group2","Group1","e_g"])["e"].mean()

In [ ]:
df.groupby(["Group2","Group1","e_g"])["e"].count()/df.groupby(["Group2","Group1"])["e"].count()

In [ ]:
sns.boxplot(data=df[df["Group2"]=="LAMP1"], x="Group1",y="e")